# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
create_statement =  """CREATE TABLE IF NOT EXISTS songdatasession (session_id INT,
        itemInSession INT,
        artist_name TEXT,
        song_title TEXT,
        song_length DOUBLE,
        PRIMARY KEY ((session_id, itemInSession)))"""
session.execute(create_statement)

select_example = "SELECT artist_name, song_title, song_length FROM songdatasession WHERE session_id = %s AND itemInSession =  %s"
                    

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songdatasession (session_id, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES  (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
result = session.execute(select_example, (338, 4))

for row in result:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


The result above shows that the query is working as expected and it helps us find the artist name and song title and song length for a specific session id

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
create_statement2 =  """CREATE TABLE IF NOT EXISTS artistdatasession1 (
        userid INT,
        session_id INT,
        itemInSession INT,
        artist_name TEXT,
        song_title TEXT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((userid, session_id), itemInSession))"""
session.execute(create_statement2)
select_example2 = "SELECT artist_name, song_title, firstName, lastName FROM artistdatasession1 WHERE userid =  %s and session_id = %s"

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
                query = "INSERT INTO artistdatasession1 (userid, session_id, itemInSession, artist_name,  song_title, firstName, lastName)"
        query = query + "VALUES  (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], (line[4])))

The second query below selects the artist_name, song_title, firstname and lastname for any given userid

In [15]:
select_example2 = "SELECT artist_name, song_title, firstName, lastName FROM artistdatasession1 WHERE userid =  %s and session_id = %s"

In [16]:
result2 = session.execute(select_example2, (10, 182))

for row in result2:
    print (row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


The results above show the user's first name and last name and all the songs that he played in session 182

In [17]:
create_statement3 =  """CREATE TABLE IF NOT EXISTS artistdatasongs1 (
        song_title TEXT,
        userid INT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY ((song_title), userid))"""
session.execute(create_statement3)
                    

In [22]:
select_example3 = "SELECT firstName, lastName FROM artistdatasongs1 WHERE song_title = %s"

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO artistdatasongs1 (song_title, userid, firstName, lastName)"
        query = query + "VALUES  (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], (line[4])))

The output below shows the first and last names of all users who played the song 'All Hands Against His Own'

In [24]:
result3 = session.execute(select_example3, ('All Hands Against His Own',))

for row in result3:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
##Droping the tables after use
drop_songdatasession =  "DROP TABLE songdatasession"
drop_artistdatasession1 = "DROP TABLE artistdatasession1"
user_data_by_song_title = "DROP TABLE user_data_by_song_title"

In [ ]:
session.execute(drop_song_data_by_session)
session.execute(song_user_data_by_user_and_session_data)
session.execute(user_data_by_song_title)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()